In [28]:
import pandas as pd
import numpy as np
import os

# Data

## Old

In [29]:
stock63 = pd.read_csv('./data/firstCl_stock63.csv')
stock64 = pd.read_csv('./data/firstCl_stock64.csv').drop(columns=['ราคา/Kg'])
stock65 = pd.read_csv('./data/firstCl_stock65.csv').drop(columns=['ราคา/Kg'])
stock66 = pd.read_csv('./data/firstCl_stock66.csv').drop(columns=['หักแกน/Kg.','วันที่คอยล์หมดม้วน','หมายเหตุ'])
stock67 = pd.read_csv('./data/firstCl_stock67.csv').drop(columns=['หักแกน/Kg.','วันที่คอยล์หมดม้วน','หมายเหตุ'])

In [30]:
stock62_color = pd.read_csv('./color/Stock_Coil(62Color).csv')
stock63_color = pd.read_csv('./color/Stock_Coil(63Color).csv')
stock64_color = pd.read_csv('./color/Stock_Coil(64Color).csv')
stock65_color = pd.read_csv('./color/Stock_Coil(65Color).csv')
stock66_color = pd.read_csv('./color/Stock_Coil(66Color).csv')
stock67_color = pd.read_csv('./color/Stock_Coil(67Color).csv')

In [31]:
stock63 = pd.merge(stock63,stock63_color,on='Describtion').drop(columns=['Colour'])
stock64 = pd.merge(stock64,stock64_color,on='Describtion').drop(columns=['Colour'])
stock65 = pd.merge(stock65,stock65_color,on='Describtion').drop(columns=['Colour'])
stock66 = pd.merge(stock66,stock66_color,on='Describtion').drop(columns=['Colour'])
stock67 = pd.merge(stock67,stock67_color,on='Describtion').drop(columns=['Colour'])

### Clean Old

In [32]:
def fill_null_values(df):
    for col in df.columns:
        dtype = df[col].dtype

        # Handle object columns (e.g., strings)
        if dtype == 'object':
            # Fill null values first
            df[col] = df[col].fillna('Unknown')

            # Then convert to categorical with 'Unknown' category
            df[col] = df[col].astype('category')

        # Handle float columns
        elif pd.api.types.is_numeric_dtype(dtype) and dtype.name == 'float64':
            df[col] = df[col].fillna(df[col].mean())

    return df

In [33]:
# clean data function
def clean(df):
    df = df.astype(str).apply(lambda x: x.str.replace(',', ''))
    df = df.astype(str).apply(lambda x: x.str.replace(' ', '' ))
    
    df['Weight (kg.)'] = (df['Weight (kg.)'].replace('', np.nan)).astype(float)
    df['Length (m.)'] = df['Length (m.)'].astype(float)
    df['คงเหลือ/kg.'] = (df['คงเหลือ/kg.'].replace('', np.nan)).astype(float)
    #df['คงเหลือ/m.'] = (df['คงเหลือ/m.'].replace('#VALUE!', np.nan)).astype(float)
    
    df['ยี่ห้อ'] = df['ยี่ห้อ'].astype('category')
    df['AZ'] = df['AZ'].astype('category')
    df['ลูกที่'] = df['ลูกที่'].astype(float)
    df['kg./m.'] = df['kg./m.'].astype(float)
    df['thickness'] = df['thickness'].astype('category')
    df['Colors'] = df['Colors'].astype('category')
    
    df['date'] = pd.to_datetime(df['date'])
    df['day'] = df['date'].dt.day
    df['month'] = df['date'].dt.month
    #df['year'] = df['date'].dt.year

    df = fill_null_values(df)
    # target
    # in precentage of คงเหลือ over Weight
    df['target'] = df['คงเหลือ/kg.'] / df['Weight (kg.)'] * 100
    #df['target2'] = df['คงเหลือ/m.'] / df['Length (m.)']
    
    df = df.drop(columns=['date','คงเหลือ/kg.','คงเหลือ/m.'])
    df = df.dropna(subset=['target'])
    
    return df

In [34]:
# feature extracrtion function
def feature_ex(df):
    return df

In [35]:
# train dataframe 
train = pd.concat([stock63, stock64, stock65,stock66])
train =  clean(train)
train

/var/folders/9q/gllpszsd7tx204064mvtm3540000gn/T/ipykernel_22316/1208771839.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])


,Describtion,ยี่ห้อ,AZ,ลูกที่,thickness,Weight (kg.),Length (m.),kg./m.,Colors,day,month,target
0,CoilTX190417B19,??,nan,1.0,0.2,3782.0,2851.0,1.33,Zinc,17.000000,6.000000,0.000000
1,CoilWSA1811150664,??,nan,2.0,0.2,3960.0,3023.0,1.31,Zinc,10.000000,8.000000,1.161616
2,CoilA200601C15,DM,nan,1.0,0.2,3508.0,2660.0,1.32,Zinc,31.000000,7.000000,0.000000
3,CoilA200602B3,DM,nan,2.0,0.2,2644.0,2024.0,1.31,Zinc,29.000000,9.000000,0.000000
4,CoilDDVL0367#P94,??,nan,3.0,0.2,4209.0,3142.0,1.34,Zinc,16.494675,6.373802,97.457828
...,...,...,...,...,...,...,...,...,...,...,...,...
451,CoilDM53BFB230924A03B,DM,nan,9.0,0.3,3958.0,2077.0,1.91,GlossyGray,28.000000,11.000000,27.438100
452,CoilDM55BFB231018A02B,DM,nan,10.0,0.3,3790.0,1988.0,1.91,GlossyGray,15.000000,12.000000,100.000000
453,Coil230621C16C,DM,nan,1.0,0.35,2740.0,1237.0,2.22,woodsoft,8.000000,8.000000,80.875912
454,Coil230621D06C,DM,nan,1.0,0.35,2638.0,1190.0,2.22,DarkWoodGrain,8.000000,8.000000,0.000000


In [36]:
len(train['Describtion'].unique())

1267

In [37]:
# test dataframe
test = clean(stock67)
test

/var/folders/9q/gllpszsd7tx204064mvtm3540000gn/T/ipykernel_22316/1208771839.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])


,Describtion,ยี่ห้อ,AZ,ลูกที่,thickness,Weight (kg.),Length (m.),kg./m.,Colors,day,month,target
0,CoilGL9-TL230922B2357T,TD,nan,1.0,0.2,3228.0,2672.0,1.21,Zinc,24.0,10.0,2.230483
1,CoilGP1B07230652-C,DM,nan,1.0,0.23,3720.0,2513.0,1.48,Zinc,18.0,4.0,0.000000
2,CoilGL13-TL231026B2449,DM,nan,2.0,0.23,3890.0,2903.0,1.34,Zinc,16.0,1.0,7.403599
3,CoilGK1-DFB231115042,DM,nan,1.0,0.25,3628.0,2312.0,1.57,Zinc,4.0,6.0,1.653804
4,CoilGL7-230807A554T,TD,nan,1.0,0.28,3898.0,2340.0,1.67,Zinc,9.0,12.0,2.514110
...,...,...,...,...,...,...,...,...,...,...,...,...
300,Coil230621C16C,DM,nan,1.0,0.35,2740.0,1237.0,2.22,woodsoft,8.0,8.0,0.000000
301,Coil230809D08B,DM,nan,2.0,0.35,2754.0,1259.0,2.19,DarkWoodGrain,11.0,8.0,21.568627
302,CoilDM54BFB231009A55A,DM,nan,1.0,0.3,4092.0,2155.0,1.90,Purple,27.0,1.0,60.899316
303,Coil2302CO04022-AJ3,TT,nan,1.0,0.35,1820.0,812.0,2.24,coffeeoakwood,7.0,9.0,100.000000


# New

In [38]:
file_name = os.listdir('./new_data')
first = pd.read_csv(f'./new_data/{file_name[0]}', encoding='cp1252')
second = pd.read_csv(f'./new_data/{file_name[1]}', encoding='cp1252')
third = pd.read_csv(f'./new_data/{file_name[2]}', encoding='cp1252')
forth = pd.read_csv(f'./new_data/{file_name[3]}', encoding='cp1252')
fifth = pd.read_csv(f'./new_data/{file_name[4]}', encoding='cp1252')

In [39]:
def average_dicts(dicts):
  averaged_dict = {}
  for key in set(dict(dicts[0]).keys() | dict(dicts[1]).keys() | dict(dicts[2]).keys()):
    values = [d.get(key, 0) for d in dicts]
    averaged_dict[key] = sum(values) / len(values)
  return averaged_dict


In [40]:
averaged_dict = average_dicts([dict(first['colorT'].value_counts()),
                               dict(second['colorT'].value_counts()),
                               dict(third['colorT'].value_counts()),
                               dict(forth['colorT'].value_counts()),
                               dict(fifth['colorT'].value_counts()),])

In [41]:
train['Bias'] = train['Colors'].map(averaged_dict).fillna(0)
train

,Describtion,ยี่ห้อ,AZ,ลูกที่,thickness,Weight (kg.),Length (m.),kg./m.,Colors,day,month,target,Bias
0,CoilTX190417B19,??,nan,1.0,0.2,3782.0,2851.0,1.33,Zinc,17.000000,6.000000,0.000000,9.8
1,CoilWSA1811150664,??,nan,2.0,0.2,3960.0,3023.0,1.31,Zinc,10.000000,8.000000,1.161616,9.8
2,CoilA200601C15,DM,nan,1.0,0.2,3508.0,2660.0,1.32,Zinc,31.000000,7.000000,0.000000,9.8
3,CoilA200602B3,DM,nan,2.0,0.2,2644.0,2024.0,1.31,Zinc,29.000000,9.000000,0.000000,9.8
4,CoilDDVL0367#P94,??,nan,3.0,0.2,4209.0,3142.0,1.34,Zinc,16.494675,6.373802,97.457828,9.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,CoilDM53BFB230924A03B,DM,nan,9.0,0.3,3958.0,2077.0,1.91,GlossyGray,28.000000,11.000000,27.438100,0.0
452,CoilDM55BFB231018A02B,DM,nan,10.0,0.3,3790.0,1988.0,1.91,GlossyGray,15.000000,12.000000,100.000000,0.0
453,Coil230621C16C,DM,nan,1.0,0.35,2740.0,1237.0,2.22,woodsoft,8.000000,8.000000,80.875912,0.8
454,Coil230621D06C,DM,nan,1.0,0.35,2638.0,1190.0,2.22,DarkWoodGrain,8.000000,8.000000,0.000000,0.0


In [42]:
test['Bias'] = test['Colors'].map(averaged_dict).fillna(0)
test

,Describtion,ยี่ห้อ,AZ,ลูกที่,thickness,Weight (kg.),Length (m.),kg./m.,Colors,day,month,target,Bias
0,CoilGL9-TL230922B2357T,TD,nan,1.0,0.2,3228.0,2672.0,1.21,Zinc,24.0,10.0,2.230483,9.8
1,CoilGP1B07230652-C,DM,nan,1.0,0.23,3720.0,2513.0,1.48,Zinc,18.0,4.0,0.000000,9.8
2,CoilGL13-TL231026B2449,DM,nan,2.0,0.23,3890.0,2903.0,1.34,Zinc,16.0,1.0,7.403599,9.8
3,CoilGK1-DFB231115042,DM,nan,1.0,0.25,3628.0,2312.0,1.57,Zinc,4.0,6.0,1.653804,9.8
4,CoilGL7-230807A554T,TD,nan,1.0,0.28,3898.0,2340.0,1.67,Zinc,9.0,12.0,2.514110,9.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,Coil230621C16C,DM,nan,1.0,0.35,2740.0,1237.0,2.22,woodsoft,8.0,8.0,0.000000,0.8
301,Coil230809D08B,DM,nan,2.0,0.35,2754.0,1259.0,2.19,DarkWoodGrain,11.0,8.0,21.568627,0.0
302,CoilDM54BFB231009A55A,DM,nan,1.0,0.3,4092.0,2155.0,1.90,Purple,27.0,1.0,60.899316,1.0
303,Coil2302CO04022-AJ3,TT,nan,1.0,0.35,1820.0,812.0,2.24,coffeeoakwood,7.0,9.0,100.000000,0.0


# Model

In [43]:
from sklearn.metrics import mean_absolute_error
from sklearn.utils import shuffle

train = shuffle(train)

In [44]:
train

,Describtion,ยี่ห้อ,AZ,ลูกที่,thickness,Weight (kg.),Length (m.),kg./m.,Colors,day,month,target,Bias
208,CoilMP1P189002,BST,nan,2.0,0.35,4692.0,2100.0,2.23,blacksand,10.0,4.0,0.000000,0.6
56,CoilA220619C12,DM,nan,4.0,0.3,4482.0,2260.0,1.98,Zinc,12.0,8.0,1.695672,9.8
195,coilMP1P160269,BST,nan,1.0,0.35,4986.0,2180.0,2.29,greengem,20.0,6.0,29.281990,0.0
182,CoilCP1P078370,BST,nan,1.0,0.35,4535.0,2007.0,2.26,FreshGreen,7.0,9.0,2.028666,0.0
461,CoilLHVB0505,CT,nan,1.0,0.3,4175.0,2168.0,1.93,skyblue,25.0,2.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,CoilSWM265-0143,SWM,nan,1.0,0.35,3430.0,1524.0,2.25,Zinc,22.0,12.0,0.000000,9.8
459,Coil22D160037,ST,nan,1.0,0.35,4113.0,1828.0,2.25,mattegray,28.0,6.0,0.000000,0.0
404,CoilLYWS0471QT,CT,nan,1.0,0.3,3373.0,1680.0,2.01,skyblue,29.0,1.0,0.000000,0.0
450,CoilMM3P027627,JJ,nan,3.0,0.35,4597.0,2062.0,2.23,Brown,8.0,7.0,1.435719,3.2


## LightGBM

In [45]:
import lightgbm as lgb

# lightgbm model 
model_lightgbm = lgb.LGBMRegressor(
    objective='regression',  # Specify regression task
    num_leaves=31,
    learning_rate=0.1,
    n_estimators=25
)

# fit the model
model_lightgbm.fit(train.drop(columns=['target', 'Describtion']), train['target'])

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000148 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 787
[LightGBM] [Info] Number of data points in the train set: 1958, number of used features: 11
[LightGBM] [Info] Start training from score 12.340598


LGBMRegressor(n_estimators=25, objective='regression')

In [46]:
# inference
pred = model_lightgbm.predict(test.drop(columns=['target', 'Describtion']))

# negative output is zero
pred = np.maximum(pred,0)

# evaluation metric
# mean absolute error
mean_absolute_error(np.array(test['target']), pred)

22.227300159022306

In [47]:
# compare output
print("pred:test['target']")
for i in range(len(pred)):
    print(i,'pred:' ,pred[i],' target: ',np.array(test['target'])[i])

pred:test['target']
0 pred: 25.347406095233016  target:  2.2304832713754648
1 pred: 5.2622944143140336  target:  0.0
2 pred: 16.726453490121628  target:  7.403598971722365
3 pred: 8.405270990013964  target:  1.6538037486218304
4 pred: 15.4081660869336  target:  2.5141097998973834
5 pred: 39.542139632634225  target:  5.034182722187694
6 pred: 7.73464841529173  target:  0.0
7 pred: 19.615533321991837  target:  6.834030683403068
8 pred: 12.529169348504087  target:  0.0
9 pred: 14.027736083407754  target:  0.0
10 pred: 19.37626013739813  target:  0.0
11 pred: 62.256716096683746  target:  0.0
12 pred: 62.256716096683746  target:  2.0715166461159065
13 pred: 8.014501990408634  target:  0.0
14 pred: 5.592592218994097  target:  0.0
15 pred: 8.992856766813386  target:  0.0
16 pred: 2.4078164483918134  target:  0.0
17 pred: 2.4078164483918134  target:  0.0
18 pred: 3.158250660256207  target:  0.0
19 pred: 3.228648403769661  target:  0.0
20 pred: 0.7892204182416065  target:  0.0
21 pred: 3.633014

## XGBRegressor

In [48]:
from xgboost import XGBRegressor
# lightgbm model 
xgb_model = XGBRegressor(n_estimators=10, learning_rate=0.1, max_depth=10, random_state=42, enable_categorical=True)

# fit the model
xgb_model.fit(train.drop(columns=['target', 'Describtion']), train['target'])

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=10, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [49]:
# inference
pred = xgb_model.predict(test.drop(columns=['target', 'Describtion']))

# negative output is zero
pred = np.maximum(pred,0)

# evaluation metric
# mean absolute error
mean_absolute_error(np.array(test['target']), pred)

21.539826804198633

In [50]:
# compare output
print("pred:test['target']")
for i in range(len(pred)):
    print(i,'pred:' ,pred[i],' target: ',np.array(test['target'])[i])

pred:test['target']
0 pred: 26.967539  target:  2.2304832713754648
1 pred: 18.135664  target:  0.0
2 pred: 24.727118  target:  7.403598971722365
3 pred: 15.600734  target:  1.6538037486218304
4 pred: 19.029415  target:  2.5141097998973834
5 pred: 29.554607  target:  5.034182722187694
6 pred: 15.600734  target:  0.0
7 pred: 15.430959  target:  6.834030683403068
8 pred: 7.2403326  target:  0.0
9 pred: 18.581387  target:  0.0
10 pred: 18.581387  target:  0.0
11 pred: 29.554607  target:  0.0
12 pred: 29.554607  target:  2.0715166461159065
13 pred: 22.619057  target:  0.0
14 pred: 22.212774  target:  0.0
15 pred: 20.718681  target:  0.0
16 pred: 17.940777  target:  0.0
17 pred: 17.940777  target:  0.0
18 pred: 19.135365  target:  0.0
19 pred: 17.182798  target:  0.0
20 pred: 14.346028  target:  0.0
21 pred: 17.971615  target:  0.0
22 pred: 14.987211  target:  2.4896265560165975
23 pred: 12.243825  target:  0.0
24 pred: 18.167576  target:  0.0
25 pred: 16.258183  target:  0.0
26 pred: 17.067

## CatBoostRegressor

In [51]:
def to_int(df):
    # Filter columns excluding target and Describtion
    filtered_cols = [col for col in df.columns if col not in ['target', 'Describtion']]

    # Convert float columns to int in the filtered list
    for col in filtered_cols:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('int')
            
    return df , filtered_cols

In [52]:
from catboost import CatBoostRegressor

catboost_model = CatBoostRegressor(iterations=100, learning_rate=0.1, depth=6,
                                   cat_features=['ยี่ห้อ', 'ลูกที่','AZ', 'thickness', 'Weight (kg.)', 'Length (m.)',
                                                'kg./m.', 'Colors', 'day', 'month', 'Bias'],
                                   random_seed=42, silent=True)

train_int,filtered_cols = to_int(train)

# fit the model
catboost_model.fit(train_int[filtered_cols], train_int['target'])

In [53]:
test_int,filtered_cols = to_int(test)
        
# inference
pred = catboost_model.predict(test_int[filtered_cols])

# negative output is zero
pred = np.maximum(pred,0)

# evaluation metric
# mean absolute error
mean_absolute_error(np.array(test_int['target']), pred)

19.100429957978015

In [54]:
# compare output
print("pred:test['target']")
for i in range(len(pred)):
    print(i,'pred:' ,pred[i],' target: ',np.array(test['target'])[i])

pred:test['target']
0 pred: 9.344727447711673  target:  2.2304832713754648
1 pred: 9.616343949224746  target:  0.0
2 pred: 12.087443031957875  target:  7.403598971722365
3 pred: 12.454077559165217  target:  1.6538037486218304
4 pred: 12.014771650803818  target:  2.5141097998973834
5 pred: 18.045821404591848  target:  5.034182722187694
6 pred: 5.367732654986539  target:  0.0
7 pred: 5.2059499068710675  target:  6.834030683403068
8 pred: 12.83465816003497  target:  0.0
9 pred: 11.844160658542357  target:  0.0
10 pred: 20.016611432543865  target:  0.0
11 pred: 3.810053167164142  target:  0.0
12 pred: 10.778358765735103  target:  2.0715166461159065
13 pred: 6.874088170727797  target:  0.0
14 pred: 16.75886627483669  target:  0.0
15 pred: 9.159473898763402  target:  0.0
16 pred: 7.740685494486783  target:  0.0
17 pred: 6.556331876530503  target:  0.0
18 pred: 14.029057290103989  target:  0.0
19 pred: 6.311017876112485  target:  0.0
20 pred: 7.451971804537445  target:  0.0
21 pred: 10.018950